In [127]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import sys

In [128]:
# UPDATE THIS
csvIndex = 0
edge_driver_path = "E:/Downloads/edgedriver_win64/msedgedriver.exe"

In [129]:
hospitals = pd.read_csv(f'hospitals_{csvIndex}.csv')
hospitals.drop(columns=['PPN / NON PPN', 'Address'], inplace=True)

In [130]:
hospitals['Rating'] = [None] * len(hospitals)
hospitals['Number of Reviews'] = [None] * len(hospitals)
hospitals['Plus Code'] = [None] * len(hospitals)

In [131]:
def getFullAddress(rowIndex):
    print("Generating full address for row", rowIndex)
    return ', '.join(hospitals.iloc[rowIndex].dropna()).replace('\n', '').replace('\r', '')

In [132]:
def searchForAddress(address, driver):
    print("Searching for address:", address)
    search_box = driver.find_element(By.ID, 'searchboxinput')
    search_box.clear() 
    search_box.send_keys(address)
    search_box.send_keys(Keys.RETURN)         

    print("Searched")

In [133]:
"""
EXAMPLE HTML
<div class="F7nice " jslog="76333;mutable:true;">
    <span>
        <span aria-hidden="true">4.8</span>
        <span class="ceNzKf" role="img" aria-label="4.8 stars ">
            <span class="rFrJzc"></span>
            <span class="rFrJzc"></span>
            <span class="rFrJzc"></span>
            <span class="rFrJzc"></span>
            <span class="rFrJzc"></span>
        </span>
    </span>
    <span>
        <span>
            <span aria-label="1,168 reviews">(1,168)</span>
        </span>
    </span>
</div>
"""

def scrape_data(driver):
    time.sleep(2)
    print("Scraping data")
    if driver.current_url.startswith("https://www.google.com/maps/search/"):
        print("In search page")
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a.hfpxzc"))
            ).click()
        except Exception as e:
            print("No search results found:", e)
            return None, None, None


    def get_element_text(selector, attribute=None):
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, selector))
            )
            return element.get_attribute(attribute) if attribute else element.text
        except Exception as e:
            print(f"Error scraping {selector}:", e)
            return None

    rating = get_element_text("div.F7nice span[aria-hidden='true']")
    num_reviews = get_element_text("div.F7nice span > span > span[aria-label]", "aria-label")
    if num_reviews:
        num_reviews = int(num_reviews.strip('()').replace(',', '').split(' ')[0])
    plus_code = get_element_text("button.CsEnBe[aria-label*='Plus code: ']", "aria-label")
    if plus_code:
        plus_code = plus_code.split(': ')[1]
    
    print("Scraped data:", rating, num_reviews, plus_code)
    return rating, num_reviews, plus_code




In [134]:
service = Service(edge_driver_path)
driver = webdriver.Edge(service=service)

driver.get('https://www.google.com/maps')
with open(f"error_log{csvIndex}.txt", "w") as f:
    for i in range(len(hospitals)):
        try:
            print(f"Processing row {i}")
            address = getFullAddress(i)
            searchForAddress(address, driver)


            rating, num_reviews, plus_code = scrape_data(driver)

            hospitals.at[i, 'Rating'] = rating
            hospitals.at[i, 'Number of Reviews'] = num_reviews
            hospitals.at[i, 'Plus Code'] = plus_code

            close_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Close']"))
            )
            close_button.click()
            time.sleep(2)  

            if i % 10 == 0:
                hospitals.to_csv(f'hospitals_{csvIndex}_ratings.csv', index=False)

        except Exception as e:
            f.write(f"Error at index {i}: {str(e)}\n")
            f.flush()
            continue
        
driver.quit()

Processing row 11
Generating full address for row 11
Searching for address: Sravani Hospital, GUNTUR, ANDHRA PRADESH
Searched
Scraping data
In search page
Scraped data: 4.3 439 7FW2+PH Guntur, Andhra Pradesh
Processing row 12
Generating full address for row 12
Searching for address: Vedanta Hospitals, GUNTUR, ANDHRA PRADESH
Searched
Scraping data
Scraped data: 4.0 392 8F47+VC Guntur, Andhra Pradesh
Processing row 13
Generating full address for row 13
Searching for address: NRI General Hospital, GUNTUR, ANDHRA PRADESH
Searched
Scraping data
Error scraping div.F7nice span > span > span[aria-label]: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=130.0.2849.56)
Stacktrace:
	GetHandleVerifier [0x00007FF66594DC75+12853]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF665BF83C4+2250276]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF665B376A6+1460486]
	(No symbol) [0x00007FF6656E84C